<a href="https://colab.research.google.com/github/mulaab/Outlier-Detection/blob/main/Evaluation_for_Unsupervised_outlier_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this topic, we present about the measures for anomaly detection if we use method unsupervised outlier detection. We know that unsupervised outlier detection using  data without label. We will show one methods for it which em mv mehods. the first step we install pyod and  and emmv library with command pip install pyod and pip install emmv

> ! pip install pyod 

>  ! pip install emmv block

>  !pip list -v for know list of packages in colab




In [ ]:
!pip install emmv

In [ ]:
!pip install pyod

In [ ]:
!pip install adtk

     |████████████████████████████████| 60 kB 3.3 MB/s 


In [ ]:
!pip install combo

  Created wheel for combo: filename=combo-0.1.2-py3-none-any.whl size=42025 sha256=1b8c9f4924310e53d4477b7748025f74652ed4dc593f20de76dd64d69edcea10
  Stored in directory: /root/.cache/pip/wheels/cf/2e/45/d4cb985fb061e3ab636d350b76114d2639d84eab16225c7776
Successfully built combo


In [ ]:
import numpy as np
from pyod.models.copod import COPOD

from emmv import emmv_scores

rng = np.random.RandomState(42)

NUM_COLS = 7
# Generate train data
X = 0.3 * rng.randn(100, NUM_COLS)
X_train = np.r_[X + 2, X - 2]
# Generate some regular novel observations
X = 0.3 * rng.randn(20, NUM_COLS)
X_regular = np.r_[X + 2, X - 2]
# Generate some abnormal novel observations
X_outliers = rng.uniform(low=-4, high=4, size=(20, NUM_COLS))
# fit the model
model = COPOD()
model.fit(X_train)

# Get EM & MV scores
X_test = np.concatenate((X_regular, X_outliers), axis=0)
test_scores = emmv_scores(model, X_test)
print('Excess Mass score;', test_scores['em'])
print('Mass Volume score:', test_scores['mv'])

Excess Mass score; 0.0072596801
Mass Volume score: 1067810.6455843074


In [ ]:
import pandas as pd
import numpy as np
from adtk.data import validate_series
from adtk.detector import GeneralizedESDTestAD

from emmv import emmv_scores

rng = np.random.RandomState(42)

# Generate train data
timestamps = pd.date_range("2018-01-01", periods=200, freq="H")
X = 0.3 * rng.randn(100)
values = np.r_[X + 2, X - 2]
data = pd.Series(values, index=timestamps)
X_train = validate_series(data)

# Generate some regular novel observations
X = 0.3 * rng.randn(67)
X_regular = np.r_[X + 2, X - 2]
# Generate some abnormal novel observations
X_outliers = rng.uniform(low=-4, high=4, size=(66))
# Create test data
timestamps = pd.date_range("2018-01-01", periods=200, freq="H")
values = np.concatenate((X_regular, X_outliers), axis=0)
data = pd.Series(values, index=timestamps)
X_test = validate_series(data)

# Fit model
model = GeneralizedESDTestAD()
anomalies = model.fit_detect(X_train)

# Get EM & MV scores

# TF models do not have a "decision_function" method, so we need to specify
# our own custom anomaly scoring function. This one is specific to GeneralizedESDTestAD.
# It is adapted from: https://github.com/odnura/adtk/blob/73bfb30ba457dd540e8aea82782431254da480ce/src/adtk/detector/_detector_1d.py#L346
def scoring_function(model, df):
    s = pd.Series(df) # 1D data expected
    new_sum = s + model._normal_sum
    new_count = model._normal_count + 1
    new_mean = new_sum / new_count
    new_squared_sum = s ** 2 + model._normal_squared_sum
    new_std = np.sqrt(
        (
            new_squared_sum
            - 2 * new_mean * new_sum
            + new_count * new_mean ** 2
        )
        / (new_count - 1)
    )
    anomaly_scores = (s - new_mean).abs() / new_std
    return anomaly_scores

test_scores = emmv_scores(model, X_test, scoring_function)
print('Excess Mass score;', test_scores['em'])
print('Mass Volume score:', test_scores['mv'])

Excess Mass score; 0.006291853399999999
Mass Volume score: 6.5658257289213795


In [ ]:
'''
Author: Christian O'Leary
Email: cjjoleary@gmail.com
'''

import os
import random

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.models import Sequential

from emmv import emmv_scores

seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
rng = np.random.RandomState(seed)
tf.random.set_seed(seed)
NUM_COLS = 1

# Generate train data
X = 0.3 * rng.randn(100, NUM_COLS)
X_train = np.r_[X + 2, X - 2]
# Generate some regular novel observations
X = 0.3 * rng.randn(20, NUM_COLS)
X_regular = np.r_[X + 2, X - 2]
# Generate some abnormal novel observations
X_outliers = rng.uniform(low=-4, high=4, size=(20, NUM_COLS))
X_test = np.concatenate((X_regular, X_outliers), axis=0)

# fit the model
model = Sequential([
            InputLayer(input_shape=NUM_COLS),
            Dense(32),
            Dense(NUM_COLS, activation='relu')
        ])
model.compile(loss='mse', optimizer='adam')
model.fit(
    X_train,
    X_train, # i.e. reconstruction model
    validation_split=0.1,
    epochs=20,
    batch_size=64,
    verbose=1
)

# Get EM & MV scores

# TF models do not have a "decision_function" method, so we need to specify
# our own custom anomaly scoring function. This one uses MAPE.
def scoring_function(model, df):
    offset = 0.00000001 # to prevent division by 0
    # 1. model predictions
    preds = model.predict(df)
    # 2. Use a regression metric, e.g. MAPE
    anomaly_scores = np.mean((np.abs(preds - df) / (df + offset)), axis=1) # i.e. anomaly score
    return anomaly_scores

test_scores = emmv_scores(model, X_test, scoring_function)
print('Excess Mass score;', test_scores['em'])
print('Mass Volume score:', test_scores['mv'])

Epoch 1/20
3/3 [==============================] - 0s 64ms/step - loss: 3.1599 - val_loss: 4.0695
Epoch 2/20
3/3 [==============================] - 0s 15ms/step - loss: 2.9964 - val_loss: 4.0695
Epoch 3/20
3/3 [==============================] - 0s 15ms/step - loss: 2.8433 - val_loss: 4.0695
Epoch 4/20
3/3 [==============================] - 0s 14ms/step - loss: 2.7012 - val_loss: 4.0695
Epoch 5/20
3/3 [==============================] - 0s 12ms/step - loss: 2.5698 - val_loss: 4.0695
Epoch 6/20
3/3 [==============================] - 0s 15ms/step - loss: 2.4601 - val_loss: 4.0695
Epoch 7/20
3/3 [==============================] - 0s 11ms/step - loss: 2.3511 - val_loss: 4.0695
Epoch 8/20
3/3 [==============================] - 0s 10ms/step - loss: 2.2648 - val_loss: 4.0695
Epoch 9/20
3/3 [==============================] - 0s 16ms/step - loss: 2.1843 - val_loss: 4.0695
Epoch 10/20
3/3 [==============================] - 0s 12ms/step - loss: 2.1174 - val_loss: 4.0695
Epoch 11/20
3/3 [============

In [ ]:
'''
Author: Christian O'Leary
Email: cjjoleary@gmail.com
'''

from emmv import emmv_scores

# Adapted from https://pycaret.org/setup/
# Importing dataset
from pycaret.datasets import get_data
anomalies = get_data('anomaly')

# Importing module and initializing setup
from pycaret.anomaly import *
anomaly_setup = setup(data=anomalies)

# create a model
model = create_model('iforest')
results = assign_model(model)

# Get EM & MV scores
test_scores = emmv_scores(model, anomalies)
print('Excess Mass score;', test_scores['em'])
print('Mass Volume score:', test_scores['mv'])